<a href="https://colab.research.google.com/github/bforsbe/SK2534/blob/main/Boltzmann_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the Boltzmann Distribution through Molecular Dynamics

This notebook demonstrates fundamental concepts in statistical mechanics through a simple 2D molecular dynamics simulation. By watching how energy distributes among particles in a confined system, we'll explore key concepts like:

- **Microstates vs Macrostates**: Individual particle configurations vs observable bulk properties
- **Energy Distribution**: How kinetic energy spreads among particles over time
- **Equilibrium**: The tendency of isolated systems to reach stable energy distributions
- **Temperature**: As a measure of average kinetic energy per particle

## Learning Objectives

By the end of this simulation, you should understand:
1. How individual particle motions (microstates) give rise to observable properties (macrostates)
2. Why energy naturally distributes according to statistical laws
3. The connection between molecular motion and temperature
4. How collisions lead to energy equilibration

## The Simulation Setup

We'll simulate **hard sphere collisions** in a 2D box:
- Particles bounce elastically off walls and each other
- Energy is conserved (no friction)
- We can optionally "thermostat" the system to maintain constant total energy

### Key Parameters to Experiment With:

**Think about this:**
- What happens if you increase/decrease the number of particles?
- How does the box size affect the collision rate?
- What role does the rescaling play in maintaining equilibrium?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
# -------------------
# Parameters
# -------------------
n_balls = 15          # Number of particles
box_size = 2          # Size of the simulation box
radius = 0.1          # Particle radius
dt = 0.02            # Time step
n_steps = 400        # Number of simulation steps
rescale_interval = 50 # How often to rescale velocities (0 = never)
elastic_loss = 0     # Energy loss per collision (0 = perfectly elastic)

## Running the Simulation

The simulation consists of several key components:
1. **Initialization**: Set up random positions and give one particle initial velocity
2. **Physics Loop**: Handle collisions and update positions
3. **Visualization**: Animate the results

In [ ]:
# -------------------
# Initialization
# -------------------
np.random.seed(0)
pos = np.random.rand(n_balls, 2) * (box_size - 2*radius) + radius
vel = np.zeros((n_balls, 2))
vel[0] = np.array([3.0, 2.0])  # give one ball initial velocity

# Storage
pos_history = []
vel_history = []
ke_history = []

In [ ]:
# -------------------
# Helper functions
# -------------------
def kinetic_energy(v):
    return 0.5 * np.sum(v**2, axis=1)

def rescale_velocities(vel, target_energy):
    current_energy = np.sum(kinetic_energy(vel))
    factor = np.sqrt(target_energy / current_energy)
    return vel * factor

def get_markersize(diam, plot_width, figsize):
    """Get scatter plot marker size for a diameter.

    Args:
        diam (float): diameter of the marker
        plot_width (float): width of the plot
        figsize (int, optional): size of the fig. Defaults to 8.

    Returns:
        float: marker size in points
    """
    points_whole_ax = figsize * 0.8 * 72    # 1 point = dpi / 72 pixels
    s = (diam / plot_width * points_whole_ax)**2
    return s

In [ ]:
# -------------------
# Simulation loop
# -------------------
target_energy = np.sum(kinetic_energy(vel))

for step in range(n_steps):
    # Update positions
    pos += vel * dt

    # Wall collisions
    for i in range(n_balls):
        for d in range(2):
            if pos[i, d] - radius < 0:
                pos[i, d] = radius
                vel[i, d] *= -1
                if elastic_loss > 0:
                    vel[i, d] *= 1-elastic_loss
            elif pos[i, d] + radius > box_size:
                pos[i, d] = box_size - radius
                vel[i, d] *= -1
                if elastic_loss > 0:
                    vel[i, d] *= 1-elastic_loss


    # Ball-ball collisions (elastic 2D)
    for i in range(n_balls):
        for j in range(i+1, n_balls):
            dp = pos[i] - pos[j]
            dist = np.linalg.norm(dp)
            if dist < 2*radius:  # overlap => collision
                dv = vel[i] - vel[j]
                dp_norm = dp / dist
                v_rel = np.dot(dv, dp_norm)
                if v_rel < 0:  # moving toward each other
                    impulse = v_rel * dp_norm
                    vel[i] -= impulse
                    vel[j] += impulse
                if elastic_loss > 0:
                    vel[i] *= 1-elastic_loss
                    vel[j] *= 1-elastic_loss

    # Rescale every few steps
    if rescale_interval > 0 and step % rescale_interval == 0:
        vel = rescale_velocities(vel, target_energy)

    # Save history
    pos_history.append(pos.copy())
    vel_history.append(vel.copy())
    ke_history.append(kinetic_energy(vel))

pos_history = np.array(pos_history)
vel_history = np.array(vel_history)
vel_mag_history = np.linalg.norm(vel_history, axis=2)
ke_history = np.array(ke_history)

print(f"Simulation complete! Ran for {n_steps} steps with {n_balls} particles.")
print(f"Initial total energy: {target_energy:.3f}")
print(f"Final total energy: {np.sum(ke_history[-1]):.3f}")

## Visualizing the Results

### What to Observe:

**Left Panel: Particle Motion (Microstates)**
- Watch how particles move and collide
- Notice how energy spreads from the initial fast particle to others
- Each specific arrangement of particles is a **microstate**

**Right Panel: Energy Evolution (Macrostates)**
- **Black line**: Total kinetic energy of the system
- **Red line**: Average speed of particles

**🔍 Key Questions to Consider:**
1. How does energy distribute among particles over time?
2. What happens to the total energy? (Should it be conserved?)
3. How does the average speed evolve?

In [ ]:
# -------------------
# Animation
# -------------------
fsize=5

fig, ax = plt.subplots(1,2,figsize=(2*fsize,fsize))

# Left panel: particle positions
ax[0].set_xlim(0, box_size)
ax[0].set_ylim(0, box_size)
ax[0].set_xlabel('X Position')
ax[0].set_ylabel('Y Position')
ax[0].set_title('Particle Motion')
ax[0].set_aspect('equal')
scat = ax[0].scatter(pos_history[0,:,0], pos_history[0,:,1],
                     s=get_markersize(radius*2, box_size, fsize),
                     alpha=0.7)

# Right panel: energy evolution
ax[1].set_xlim(0, n_steps)
ax[1].set_ylim(0, np.max([target_energy*1.4,np.max(vel_mag_history)*1.4]))
ax[1].set_xlabel('Time Step')
ax[1].set_ylabel('Energy / Speed')
ax[1].set_title('Energy Evolution')
grph1, = ax[1].plot(ke_history[:1].sum(axis=1),'k-', linewidth=2, label='Total KE')
grph2, = ax[1].plot(vel_mag_history[:1].mean(axis=1),'r-', linewidth=2, label='Avg Speed')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

def update(frame):
    scat.set_offsets(pos_history[frame])
    grph1.set_xdata(np.arange(frame+1))
    grph1.set_ydata(ke_history[:frame+1].sum(axis=1))
    grph2.set_xdata(np.arange(frame+1))
    grph2.set_ydata(vel_mag_history[:frame+1].mean(axis=1))
    return scat, grph1, grph2

ani = FuncAnimation(fig, update, frames=400, interval=50, blit=True)
plt.tight_layout()
plt.close(fig)
HTML(ani.to_jshtml())

## Experiments to Try

Now that you've seen the basic simulation, try modifying the parameters above and re-running to explore different scenarios:

### Experiment 1: Effect of Particle Number
Change `n_balls` to different values (5, 10, 25, 50). How does this affect:
- The time to reach equilibrium?
- The fluctuations in total energy?

### Experiment 2: Thermostat Effect  
Try different `rescale_interval` values:
- `rescale_interval = 0` (no rescaling - true microcanonical ensemble)
- `rescale_interval = 10` (frequent rescaling - canonical-like behavior)

### Experiment 3: Energy Loss
Set `elastic_loss = 0.05`. What happens to the system over time?

### Experiment 4: Initial Conditions
Try giving multiple particles initial velocities by modifying the initialization:
```python
vel[0] = np.array([3.0, 2.0])
vel[1] = np.array([-2.0, 1.0])
vel[2] = np.array([1.0, -3.0])
```

## Additional Analysis

Let's look at some static plots to better understand the energy distribution:

In [ ]:
# Plot energy evolution over time
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Total energy conservation
total_energy = ke_history.sum(axis=1)
ax1.plot(total_energy, 'k-', linewidth=2)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Total Kinetic Energy')
ax1.set_title('Energy Conservation')
ax1.grid(True, alpha=0.3)

# Energy distribution at different times
times_to_plot = [0, 50, 100, 200, -1]
colors = ['red', 'orange', 'green', 'blue', 'purple']

for i, (t, color) in enumerate(zip(times_to_plot, colors)):
    if t == -1:
        energies = ke_history[-1]
        label = f'Final (t={n_steps-1})'
    else:
        energies = ke_history[t]
        label = f't={t}'

    ax2.hist(energies, bins=10, alpha=0.6, color=color, label=label, density=True)

ax2.set_xlabel('Kinetic Energy per Particle')
ax2.set_ylabel('Probability Density')
ax2.set_title('Energy Distribution Evolution')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Energy conservation check:")
print(f"Initial total energy: {total_energy[0]:.4f}")
print(f"Final total energy: {total_energy[-1]:.4f}")
print(f"Relative change: {(total_energy[-1] - total_energy[0])/total_energy[0]*100:.2f}%")

## Connecting to Statistical Mechanics

### Microstates vs Macrostates
- **Microstate**: The exact position and velocity of every particle at a given time
- **Macrostate**: Observable quantities like total energy, average speed, temperature

### Energy Distribution and Temperature
In a real gas, the Boltzmann distribution tells us the probability of finding a particle with energy E:

$$P(E) \propto e^{-E/k_BT}$$

Where:
- $k_B$ is Boltzmann's constant
- $T$ is temperature
- Higher temperature = broader energy distribution

### Entropy and Equilibrium
- The system naturally evolves toward states with maximum entropy
- Equilibrium represents the most probable macrostate
- Individual microstates are constantly changing, but macroscopic properties stabilize

## Discussion Questions

1. **Equilibration Time**: How long does it take for energy to distribute evenly? What factors affect this?

2. **Reversibility**: Is this process reversible? Could all energy spontaneously return to one particle?

3. **Temperature**: If temperature is proportional to average kinetic energy, how would you define temperature in this system?

4. **Real vs Ideal**: How does this simplified model compare to real gas behavior?

5. **Scaling**: What would happen with 1000 particles? $10^{23}$ particles (like in a real gas)?

## Extensions and Challenges

### Challenge 1: Energy Distribution Histogram
Create a histogram showing the distribution of kinetic energies at different times. Does it approach a Boltzmann distribution?

### Challenge 2: Temperature Measurement  
Calculate an effective "temperature" from the average kinetic energy. Plot how temperature changes over time.

### Challenge 3: Pressure Calculation
Count wall collisions and calculate the pressure exerted by the gas on the container walls.

### Challenge 4: Maxwell-Boltzmann Speeds
Plot the speed distribution and compare to the theoretical Maxwell-Boltzmann distribution.

In [ ]:
# Space for your experiments and extensions!
# Try modifying the parameters above and exploring different scenarios

---

*💡 **Remember**: This simple simulation captures the essence of how microscopic chaos leads to macroscopic order - the foundation of statistical mechanics and our understanding of temperature, pressure, and entropy!*